In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"

spark = SparkSession.builder \
    .appName("jieba_test2") \
    .master("spark://node01:10077") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/18 10:13:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/18 10:13:16 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
import jieba
import jieba.analyse

stop_words_rdd = sc.textFile("hdfs:///user/spark_temp/stopwords.dat")
stop_words_set = set(stop_words_rdd.collect())

def cut_name_and_desc(item):
    id, name, desc = item
    name_cut = set(jieba.cut(name))
    name_cut_pure = set(jieba.cut(name)) - stop_words_set
    desc_cut = set(jieba.cut(desc))
    desc_cut_pure = set(jieba.cut(desc)) - stop_words_set
    return (id,name_cut,name_cut_pure,desc_cut,desc_cut_pure)

def to_count(item):
    id,name_cut,name_cut_pure,desc_cut,desc_cut_pure = item
    for i in name_cut_pure:
        yield ((id, i),1)
    # for i in desc_cut_pure:
    #     yield (i,1)

In [4]:
cur = mydb.cursor()

In [5]:
cur.reset()
cur.execute("SELECT id,name,description from items order by RAND() ")
result = cur.fetchall()
# print(result)

In [6]:
all_items_rdd = sc.parallelize(result)
# print(all_items_rdd.collect())

In [7]:

items_cut_rdd = all_items_rdd.map(cut_name_and_desc)
# print(rdd2.collect())

In [8]:
def f1(item):
    aa,c = item
    a,b = aa
    return a,b,c
word_count = items_cut_rdd.flatMap(to_count).reduceByKey(lambda a,b:a+b).map(f1)
# print(word_count.collect())

In [14]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("item_id", IntegerType(), True),
    StructField("key_word", StringType(), True),
    StructField("word_count", IntegerType(), True)
])
df1 = spark.createDataFrame(word_count, schema)
df1.createOrReplaceTempView("word_count")
df1.show()

+-------+--------+----------+
|item_id|key_word|word_count|
+-------+--------+----------+
|    319|    农庄|         1|
|    458|    喂养|         1|
|    181|      牛|         1|
|    198|      花|         1|
|    454|    岗牛|         1|
|    454|  黑白花|         1|
|    454|    喂养|         1|
|    454|    管理|         1|
|    218|      牛|         1|
|    218|      药|         1|
|    180|    母牛|         1|
|    253|      牛|         1|
|    262|      牛|         1|
|    262|    用药|         1|
|    370|    肉牛|         1|
|     85|    优质|         1|
|    417|    一号|         1|
|    355|    优质|         1|
|    132|    出售|         1|
|    132|    红牛|         1|
+-------+--------+----------+
only showing top 20 rows



In [17]:
result = spark.sql('''
    select
        item_id,
        key_word,
        sum(word_count) as word_count
    from
        word_count
    group by 
        item_id ,key_word
    order by
        word_count desc
''')
result.show()

+-------+--------+----------+
|item_id|key_word|word_count|
+-------+--------+----------+
|    445|    种苗|         1|
|     57|    特产|         1|
|    173|    配种|         1|
|    449|    岗牛|         1|
|     11|      牛|         1|
|    228|  驱虫剂|         1|
|    358|    肉牛|         1|
|    463|    岗牛|         1|
|    190|    配种|         1|
|    396|  黑白花|         1|
|    145|  黑白花|         1|
|     40|  牛肉干|         1|
|     96|    500g|         1|
|    210|荷斯坦种|         1|
|    454|    二号|         1|
|    253|    奶粉|         1|
|    416|    三号|         1|
|     13|      黑|         1|
|    209|荷斯坦种|         1|
|    130|      牛|         1|
+-------+--------+----------+
only showing top 20 rows



In [19]:
spark.sql("CREATE TABLE IF NOT EXISTS item_word_count (item_id INT, key_word STRING, word_count INT) USING hive")
result = spark.sql('''
    insert into table item_word_count select * from word_count
''')
result.show()

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);;
'CreateTable `default`.`item_word_count`, Ignore
